In [17]:
import pandas as pd
import string
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from collections import Counter

#import utk tokenizing
import nltk
nltk.download("punkt")

#install utk stemming
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [18]:
soal = pd.read_csv(r"C:\Users\DELL\Documents\Active Folder\COOLYAH\SMT 8\TA\source code\Dataset Soal C4-C6.csv")
soal.head()

,Soal Pertanyaan,Label
0,ASEAN merupakan organisasi regional Asia Tengg...,C4
1,Gerakan Non Blok merupakan organisasi negara-n...,C4
2,Salah satu penyebab kacaunya kondisi perekonom...,C4
3,Indonesia harus dapat mengatasi permasalahan e...,C5
4,Walaupun pada awalnya secara tersembunyi tujua...,C4


In [19]:
#casefold
soal["Casefold"] = soal["Soal Pertanyaan"].str.lower()
print(soal["Casefold"].head())

0    asean merupakan organisasi regional asia tengg...
1    gerakan non blok merupakan organisasi negara-n...
2    salah satu penyebab kacaunya kondisi perekonom...
3    indonesia harus dapat mengatasi permasalahan e...
4    walaupun pada awalnya secara tersembunyi tujua...
Name: Casefold, dtype: object


In [20]:
#tokenizing

#remove punctuation
def remove_punctuation(text):
  return text.translate(str.maketrans("","",string.punctuation))
soal["Tokenize"] = soal["Casefold"].apply(remove_punctuation)

#remove whitespace
def remove_whitespace_LT(text):
  return text.strip()
soal["Tokenize"] = soal["Tokenize"].apply(remove_whitespace_LT)

#NLTK word tokenize
def word_tokenize_wrapper(text):
  return word_tokenize(text)
soal["Tokenize"] = soal["Tokenize"].apply(word_tokenize_wrapper)

print(soal["Tokenize"].head())

0    [asean, merupakan, organisasi, regional, asia,...
1    [gerakan, non, blok, merupakan, organisasi, ne...
2    [salah, satu, penyebab, kacaunya, kondisi, per...
3    [indonesia, harus, dapat, mengatasi, permasala...
4    [walaupun, pada, awalnya, secara, tersembunyi,...
Name: Tokenize, dtype: object


In [21]:
#NLTK calc frequency distribution
#utk menghitung frekuensi distribusi token pada tiap row data
def freqDist_wrapper(text):
  return FreqDist(text)
soal["FreqDist"] = soal["Tokenize"].apply(freqDist_wrapper)

print(soal["FreqDist"].head().apply(lambda x: x.most_common()))

0    [(organisasi, 2), (asean, 1), (merupakan, 1), ...
1    [(organisasi, 2), (yang, 2), (gerakan, 1), (no...
2    [(indonesia, 2), (pada, 2), (awal, 2), (kemerd...
3    [(indonesia, 2), (ekonomi, 2), (pada, 2), (awa...
4    [(kebijakan, 2), (dapat, 2), (walaupun, 1), (p...
Name: FreqDist, dtype: object


In [22]:
#stopwords

#get stopword indonesia
list_stopwords = stopwords.words('indonesian')

#additional stopword manually
list_stopwords.extend(["adalah", "yaitu"])

#convert list to dictionary
list_stopwords = set(list_stopwords)

#remove stopword pd list token
def stopwords_removal(words):
  return [word for word in words if word not in list_stopwords]
soal["Stopwords"] = soal["Tokenize"].apply(stopwords_removal)

print(soal["Stopwords"].head())

0    [asean, organisasi, regional, asia, tenggara, ...
1    [gerakan, non, blok, organisasi, negaranegara,...
2    [salah, penyebab, kacaunya, kondisi, perekonom...
3    [indonesia, mengatasi, permasalahan, ekonomi, ...
4    [tersembunyi, tujuan, pemerintah, kolonial, me...
Name: Stopwords, dtype: object


In [23]:
#stemming
def stemmed_wrapper(term):
  return stemmer.stem(term)
term_dict = {}
for document in soal["Stopwords"]:
  for term in document:
    if term not in term_dict:
      term_dict[term] = ' '

print(len(term_dict))
print("------")

for term in term_dict:
  term_dict[term] = stemmed_wrapper(term)
  print(term,":",term_dict[term])

print(term_dict)
print("------")

#apply stemmed term to dataframe
def get_stemmed_term(document):
  return [term_dict[term] for term in document]
soal["Stemmed"] = soal["Stopwords"].apply(get_stemmed_term)
print(soal["Stemmed"])

500
------
asean : asean
organisasi : organisasi
regional : regional
asia : asia
tenggara : tenggara
dibentuk : bentuk
berdasarkan : dasar
deklarasi : deklarasi
bangkok : bangkok
tanggal : tanggal
8 : 8
agustus : agustus
1967 : 1967
segi : segi
politik : politik
bertujuan : tuju
gerakan : gera
non : non
blok : blok
negaranegara : negaranegara
berkembang : kembang
konferensi : konferensi
beograd : beograd
1961 : 1961
digagas : gagas
karno : karno
salah : salah
penyebab : sebab
kacaunya : kacau
kondisi : kondisi
perekonomian : ekonomi
indonesia : indonesia
kemerdekaan : merdeka
kas : kas
negara : negara
kosong : kosong
upaya : upaya
pemerintah : perintah
republik : republik
mengisi : isi
mengatasi : atas
permasalahan : masalah
ekonomi : ekonomi
dihadapi : hadap
bangsa : bangsa
perbaikan : baik
tersembunyi : sembunyi
tujuan : tuju
kolonial : kolonial
mengeluarkan : keluar
kebijakan : bijak
etis : etis
menciptakan : cipta
kalangan : kalang
pegawai : pegawai
memiliki : milik
keterampilan : 

In [24]:
#save to csv
soal.to_csv(r"C:\Users\DELL\Documents\Active Folder\COOLYAH\SMT 8\TA\source code\Dataset Soal C4-C6 preprocessed.csv",index=False,header=True)